In [22]:
import mne
import matplotlib.pyplot as plt
import os
import pandas as pd
import numpy as np
import tqdm

In [23]:
#get all subject list
participants_info = pd.read_csv('participants.tsv', sep='\t')
participants = participants_info.participant_id.tolist()

In [24]:
# get edf data for the each particpant
files = []
paths = []
for sub in participants: 
    sub_path = sub + '/eeg/'
    # get all edf files
    part_files = [fn for fn in os.listdir(sub_path) if fn.endswith('set')]
    files.append(part_files)
    paths.append(sub_path)

In [25]:
%%capture
# read raw eeglab data and append to a list
raws = []
for i in range(len(files)):
    temp = []
    for file in files[i]: 
        path = participants[i] + '/eeg/'
        raw = mne.io.read_raw_eeglab(path+file, preload=True)
        temp.append(raw)
    raws.append(temp)

In [26]:
%%capture

#remove power line noise
for r in raws: 
    for raw in r: 
        raw.notch_filter(freqs=50)

In [38]:
# test how to read in events from eeglab data files
events, dict = mne.events_from_annotations(raws[0][6])
epochs = mne.Epochs(raws[0][0], events, tmin=0.5, tmax=2.5, baseline=None)
epochs.load_data()

Used Annotations descriptions: ['BASE2T0']
Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 1 events and 321 original time points ...
0 bad epochs dropped


Number of events,1
Events,1: 1
Time range,0.500 – 2.500 sec
Baseline,off


In [75]:
# check the contents of events, useful for matching 1, 2, 3 for each run to the true label
events[0]

array([0, 0, 1])

In [65]:
# function to match numeric event labels to textual true labels
def matchLabels(events, event_dict) : 
    task_labels = list(event_dict.keys())
    labels = []

    if len(task_labels) == 3: 
        label_dict = {1:task_labels[0], 2:task_labels[1], 3:task_labels[2]}
    elif len(task_labels) == 2: 
        label_dict = {1:task_labels[0], 2:task_labels[1]}
    else: 
        label_dict = {1:task_labels[0]}
        
    for e in events: 
        # numeric event label is the last element of each row
        labels.append(label_dict[e[-1]])
    labels = np.array(labels)

    return labels

In [41]:
#raws = 109 x 14
len(raws[0])

14

In [78]:
raws = [raw for sublist in raws for raw in sublist]

In [80]:
%%capture

X_epochs = []
y_true = []
for raw in raws: 
    raw.notch_filter(freqs=50);
    
    events, event_dict = mne.events_from_annotations(raw)
    epochs = mne.Epochs(raw, events, baseline=None)
    curr_labels = matchLabels(events, event_dict)

    X_epochs.append(epochs)
    y_true.append(curr_labels)

In [ ]:
X_epochs = np.array(X_epochs).flatten()
y_true = np.array(y_true).flatten()

C:\Users\zihen\AppData\Local\Temp\ipykernel_22828\416250313.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  y_true = np.array(y_true).flatten()


In [ ]:
X_epochs.shape

(1526,)

In [ ]:
y_true.shape

(1526,)